# Main - Charcrete LCA

## 1. Set the project and import Charcrete database

In [53]:
# === Switch to kernel bw25! ===

# Brightway libraries
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import bw2analyzer as ba
import bw_processing as bwp

# General libraries
import numpy as np
import pandas as pd
from fs.zipfs import ZipFS
import json                        # Library for working with json files
from pathlib import Path           # Library for working with paths in different OS     
import matplotlib.pyplot as plt    # Library for creating plots
import os

# Our own functions
%run utils.ipynb

In [54]:
bd.projects.set_current("ei38-teaching-25")

In [55]:
wdir = os.getcwd()

### Importing Charcrete database Excel-file

In [56]:
if "Charcrete_38" in bd.databases:
    del bd.databases["Charcrete_38"]

# 1. Specify filepath to your foreground inventories.
charcrete_path = wdir+"/data/Charcrete_database_38.xlsx"
#charcrete_path = "/home/jupyter-mija.frossard/Charcrete-LCA/Water_database_38.xlsx"
# 2. Create an instance of a class that contains basic methods for importing a database from an excel file.
charcrete = bi.ExcelImporter(charcrete_path)  
# 3. `apply_strategies` is one of such basic methods, it makes sure units, locations, etc are in correct format.
charcrete.apply_strategies()
# 4. Next step is to link your foreground exchanges to existing databases by matching relevant exchanges fields.
charcrete.match_database("biosphere3", fields=("name", "unit", "categories"))
charcrete.match_database("ei 3.8 cutoff", fields=("name", "location", "unit"))
#charcrete.metadata.pop(None)  # Remove metadata None entry. TODO
# 5. If everything is linked, write database so that it is saved in your project.
if charcrete.all_linked:
    charcrete.write_database()

Extracted 2 worksheets in 0.07 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 7.15 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
7 datasets
43 exchanges
0 unlinked exchanges
  
Title: Writing activities to SQLite3 da

In [57]:
for unlinked in charcrete.unlinked:
    print(unlinked)

In [58]:
bd.databases

Databases dictionary with 7 object(s):
	Charcrete_38
	EXIOBASE 3.8.1 2017 monetary
	EXIOBASE 3.8.1 2017 monetary biosphere
	Korean Motors Chaebol
	Water_38
	biosphere3
	ei 3.8 cutoff

In [59]:
len(bd.Database("Charcrete_38"))

7

## 2. Basic calculations and comparisons

In [60]:
bd.projects.set_current("ei38-teaching-25")

In [61]:
char_db = bd.Database("Charcrete_38")

In [62]:
ei_db = bd.Database("ei 3.8 cutoff")

In [63]:
[act for act in char_db]
charcrete = [act for act in char_db if "Charcrete production 5%" in act["name"]][0]
charcrete

'Charcrete production 5%' (cubic meter, SE, None)

In [64]:
concrete_ei = [act for act in ei_db if "concrete production 20MPa" in act["name"]]
concrete_ei

['concrete production 20MPa' (cubic meter, RoW, None),
 'concrete production 20MPa' (cubic meter, ZA, None),
 'concrete production 20MPa' (cubic meter, North America without Quebec, None),
 'concrete production 20MPa' (cubic meter, CA-QC, None)]

#### Choice of an LCIA method: IPCC 2013 GWP 100

In [65]:
ipcc = ('IPCC 2013', 'climate change', 'GWP 100a')
method = bd.Method(ipcc)

#### Analysis of standard concrete production

In [66]:
concretes_gwp = [calculateLCA(act, 1, ipcc) for act in concrete_ei]

'concrete production 20MPa' (cubic meter, RoW, None) 193.85391050515568
'concrete production 20MPa' (cubic meter, ZA, None) 229.71912735575845
'concrete production 20MPa' (cubic meter, North America without Quebec, None) 217.55243665066354
'concrete production 20MPa' (cubic meter, CA-QC, None) 234.1879918934279


In [67]:
first_lca = pd.DataFrame(list(zip([act["name"] for act in concrete_ei],[act["location"] for act in concrete_ei], concretes_gwp)), columns=["activity", "location", "GWP"])
first_lca

,activity,location,GWP
0,concrete production 20MPa,RoW,193.853911
1,concrete production 20MPa,ZA,229.719127
2,concrete production 20MPa,North America without Quebec,217.552437
3,concrete production 20MPa,CA-QC,234.187992


In [68]:
best_concrete_idx = first_lca[['GWP']].idxmin()[0]
worst_concrete_idx = first_lca[['GWP']].idxmax()[0]

In [69]:
ba.print_recursive_calculation(concrete_ei[best_concrete_idx], ipcc, cutoff=0.010)

Fraction of score | Absolute score | Amount | Activity
0001 | 193.9 |     1 | 'concrete production 20MPa' (cubic meter, RoW, None)
  0.0214 | 4.153 | 858.8 | 'market for gravel, round' (kilogram, CH, None)
    0.0118 | 2.283 |  17.4 | 'market for transport, freight, lorry, unspecified' (ton kilometer, RE
  0.839 | 162.6 | 256.6 | 'market for cement, portland fly ash cement 21-35%' (kilogram, RoW, No
    00.8 |   155 | 256.6 | 'cement production, fly ash 21-35%' (kilogram, RoW, None)
      0.0296 | 5.739 | 6.118 | 'market group for electricity, medium voltage' (kilowatt hour, RAS, No
      0.146 | 28.27 | 30.21 | 'market for clinker' (kilogram, Europe without Switzerland, None)
      0.0303 | 5.868 | 6.363 | 'market for clinker' (kilogram, BR, None)
      0.129 | 24.98 | 28.23 | 'market for clinker' (kilogram, IN, None)
      0.43 | 83.42 | 90.59 | 'market for clinker' (kilogram, RoW, None)
    0.0318 | 6.173 | 44.72 | 'market for transport, freight, lorry, unspecified' (ton kilometer, 

In [70]:
ba.print_recursive_calculation(concrete_ei[worst_concrete_idx], ipcc, cutoff=0.010)

Fraction of score | Absolute score | Amount | Activity
0001 | 234.2 |     1 | 'concrete production 20MPa' (cubic meter, CA-QC, None)
  0.0483 |  11.3 |   950 | 'market for sand' (kilogram, RoW, None)
    0.0181 | 4.235 | 30.68 | 'market for transport, freight, lorry, unspecified' (ton kilometer, Ro
    0.0121 | 2.824 | 672.1 | 'gravel and sand quarry operation' (kilogram, RoW, None)
  0.0477 | 11.17 | 960.1 | 'market for gravel, round' (kilogram, RoW, None)
    0.0172 | 4.034 | 960.1 | 'gravel and sand quarry operation' (kilogram, RoW, None)
    0.0159 | 3.714 | 26.91 | 'market for transport, freight, lorry, unspecified' (ton kilometer, Ro
  0.878 | 205.6 | 239.5 | 'market for cement, Portland' (kilogram, CA-QC, None)
    0.0151 | 3.534 |  25.6 | 'market for transport, freight, lorry, unspecified' (ton kilometer, Ro
    0.855 | 200.2 | 239.5 | 'cement production, Portland' (kilogram, CA-QC, None)
      0.85 |   199 | 220.4 | 'clinker production' (kilogram, CA-QC, None)


Concrete with the least impact has "market for cement, portland fly ash cement 21-35%, RoW"

#### Analysis over the cement production

In [71]:
cement = [act for act in ei_db if 'market for cement,' in act["name"] and "kilogram" in act["unit"]]

In [72]:
#cement_gwp = [calculateLCA(act, 1, ipcc) for act in cement]
fu, data_objs, _ = bd.prepare_lca_inputs({cement[0]: 1}, method=ipcc)
lca = bc.LCA(fu, data_objs=data_objs)
lca.lci()
lca.lcia()
results_ciment = []
for act in cement:
    lca.redo_lcia({act.id: 1})
    results_ciment.append((act["name"], act["location"], lca.score))

cements_lca = pd.DataFrame(results_ciment, columns=["activity", "location", "GWP"])
cements_lca.to_csv(wdir+"/outputs/gwp_results_cement_1_kg.csv")
cements_lca

,activity,location,GWP
0,"market for cement, blast furnace slag 31-50% a...",Europe without Switzerland,0.427272
1,"market for cement, alternative constituents 45%",RoW,0.610156
2,"market for cement, pozzolana and fly ash 36-55%",RoW,0.600784
3,"market for cement, pozzolana and fly ash 6-14%",BR,0.739441
4,"market for cement, Portland",PE,0.903437
...,...,...,...
93,"market for cement, blast furnace slag 36-65%",RoW,0.588866
94,"market for cement, blast furnace slag 36-65%",ZA,0.672298
95,"market for cement, blast furnace slag 35-70%",RoW,0.445083
96,"market for cement, blast furnace slag 25-70%",RoW,0.604803


In [73]:
cements_lca.nunique()

activity    32
location    10
GWP         98
dtype: int64

#### Adding the charcrete to the comparison

In [74]:
concrete_ei.append(charcrete)
print(concrete_ei)
concretes_gwp = [calculateLCA(act, 1, ipcc) for act in concrete_ei]

['concrete production 20MPa' (cubic meter, RoW, None), 'concrete production 20MPa' (cubic meter, ZA, None), 'concrete production 20MPa' (cubic meter, North America without Quebec, None), 'concrete production 20MPa' (cubic meter, CA-QC, None), 'Charcrete production 5%' (cubic meter, SE, None)]
'concrete production 20MPa' (cubic meter, RoW, None) 193.85391050515568
'concrete production 20MPa' (cubic meter, ZA, None) 229.71912735575845
'concrete production 20MPa' (cubic meter, North America without Quebec, None) 217.55243665066354
'concrete production 20MPa' (cubic meter, CA-QC, None) 234.1879918934279
'Charcrete production 5%' (cubic meter, SE, None) 177.6456230224958


In [75]:
first_lca = pd.DataFrame(list(zip([act["name"] for act in concrete_ei],[act["location"] for act in concrete_ei], concretes_gwp)), columns=["activity", "location", "GWP"])
first_lca.to_csv(wdir+"/outputs/first_lca_results_concrete_1_m3.csv")
first_lca

,activity,location,GWP
0,concrete production 20MPa,RoW,193.853911
1,concrete production 20MPa,ZA,229.719127
2,concrete production 20MPa,North America without Quebec,217.552437
3,concrete production 20MPa,CA-QC,234.187992
4,Charcrete production 5%,SE,177.645623


In [76]:
ba.print_recursive_calculation(charcrete, ipcc, cutoff=0.010)

Fraction of score | Absolute score | Amount | Activity
0001 | 177.6 |     1 | 'Charcrete production 5%' (cubic meter, SE, None)
  0.0123 | 2.184 |  0.01 | 'Biochar production' (ton, SE, None)
  0.0346 | 6.151 |  1000 | 'market for gravel, crushed' (kilogram, CH, None)
    0.015 | 2.659 | 20.26 | 'market for transport, freight, lorry, unspecified' (ton kilometer, RE
    0.0194 | 3.437 |  1000 | 'gravel production, crushed' (kilogram, CH, None)
  0.0255 | 4.536 |   938 | 'market for sand' (kilogram, CH, None)
    0.014 | 2.494 |    19 | 'market for transport, freight, lorry, unspecified' (ton kilometer, RE
    0.0112 | 1.991 |   938 | 'gravel and sand quarry operation' (kilogram, CH, None)
  0.024 | 4.261 |   3.2 | 'market for plasticiser, for concrete, based on sulfonated melamine fo
    0.0231 |   4.1 |   3.2 | 'plasticiser production, for concrete, based on sulfonated melamine fo
      0.0121 | 2.147 | 1.075 | 'market for chemical, organic' (kilogram, GLO, None)
  0.92 | 163.5 | 186.8

**Charcrete** is not that good I guess... Need to investigate the formulation of the concrete and other parameters with more data/uncertainty analysis on : 
- cement production process and localisation
- formulation of cement/biochar/other...

## Replacing stuff in the activity

In [ ]:
replace_stuff